In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import VarianceThreshold

In [2]:
import pandas as pd

#读取训练集，此时训练集手工已去掉6个重复样本
train = pd.read_excel('Dataset/训练集.xlsx', index_col='ID')

In [3]:
testa = pd.read_excel('Dataset/测试集A.xlsx', index_col='ID')

In [4]:
data, data_a = train, testa

In [5]:
print(data.info(), data_a.info())

<class 'pandas.core.frame.DataFrame'>
Index: 794 entries, NH0995 to NH1942
Columns: 5953 entries, TOOL to Value
dtypes: float64(5183), int64(759), object(11)
memory usage: 36.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, NH1943 to NH0463
Columns: 5952 entries, TOOL to 750X1452
dtypes: float64(5289), int64(652), object(11)
memory usage: 13.6+ MB
None None


In [6]:
data = data.drop(['NH1835'], axis=0)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 5953 entries, TOOL to Value
dtypes: float64(5183), int64(759), object(11)
memory usage: 36.0+ MB
None


In [7]:
# 将训练集的文本数据转换为数值型数据
le = LabelEncoder()
data["TOOL"] = le.fit_transform(data["TOOL"])
data["Tool"] = le.fit_transform(data["Tool"])
data["TOOL_ID"] = le.fit_transform(data["TOOL_ID"])
data["Tool (#1)"] = le.fit_transform(data["Tool (#1)"])
data["TOOL (#1)"] = le.fit_transform(data["TOOL (#1)"])
data["TOOL (#2)"] = le.fit_transform(data["TOOL (#2)"])
data["Tool (#2)"] = le.fit_transform(data["Tool (#2)"])
data["Tool (#3)"] = le.fit_transform(data["Tool (#3)"])
data["Tool (#4)"] = le.fit_transform(data["Tool (#4)"])
data["Tool (#5)"] = le.fit_transform(data["Tool (#5)"])
data["TOOL (#3)"] = le.fit_transform(data["TOOL (#3)"])
data["OPERATION_ID"] = le.fit_transform(data["OPERATION_ID"])

In [8]:
# 将测试集a的文本数据转换为数值型数据
le = LabelEncoder()
data_a["TOOL"] = le.fit_transform(data_a["TOOL"])
data_a["Tool"] = le.fit_transform(data_a["Tool"])
data_a["TOOL_ID"] = le.fit_transform(data_a["TOOL_ID"])
data_a["Tool (#1)"] = le.fit_transform(data_a["Tool (#1)"])
data_a["TOOL (#1)"] = le.fit_transform(data_a["TOOL (#1)"])
data_a["TOOL (#2)"] = le.fit_transform(data_a["TOOL (#2)"])
data_a["Tool (#2)"] = le.fit_transform(data_a["Tool (#2)"])
data_a["Tool (#3)"] = le.fit_transform(data_a["Tool (#3)"])
data_a["Tool (#4)"] = le.fit_transform(data_a["Tool (#4)"])
data_a["Tool (#5)"] = le.fit_transform(data_a["Tool (#5)"])
data_a["TOOL (#3)"] = le.fit_transform(data_a["TOOL (#3)"])
data_a["OPERATION_ID"] = le.fit_transform(data_a["OPERATION_ID"])

In [9]:
# 测试集a中，缺失比例大于10%的特征，删除，其余缺失值用均值填充
for i in data_a.columns:
    average = data_a[i].mean()
    if data_a[i].count() / len(data_a) < 0.1:
        data_a = data_a.drop(i, axis=1)
    else:
        data_a[i] = data_a[i].fillna(average)

print(data_a.info())

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, NH1943 to NH0463
Columns: 5877 entries, TOOL to 750X1452
dtypes: float64(5214), int32(11), int64(652)
memory usage: 13.4+ MB
None


In [10]:
# 方差为0的特征，删除
del_var = []
for i in data.columns:
    if data[i].var() == 0:
        del_var.append(i)
data = data.drop(del_var, axis=1)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 5273 entries, TOOL to Value
dtypes: float64(4873), int32(11), int64(389)
memory usage: 31.9+ MB
None


In [11]:
# 训练集中，缺失比例大于10%的特征，删除，其余缺失值用均值填充
for i in data.columns:
    average = data[i].mean()
    if data[i].count() / len(data) < 0.1:
        data = data.drop(i, axis=1)
    else:
        data[i] = data[i].fillna(average)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 5044 entries, TOOL to Value
dtypes: float64(4644), int32(11), int64(389)
memory usage: 30.5+ MB
None


In [12]:
# 删除日期列
for i in data.columns:
    max = data[i].max()
    if max > 1e10 and (str(max).find('2016') != -1 or str(max).find('2017') != -1):
        data = data.drop(i, axis=1)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 4971 entries, TOOL to Value
dtypes: float64(4599), int32(11), int64(361)
memory usage: 30.0+ MB
None


In [13]:
#特征间相关性（线性化）过高的特征，只保留一条以显示信息，其余删除；
cor = []
tmp = []
matrix = data.corr()

for i in matrix.columns:
    if i in matrix.columns:
        index1 = matrix[matrix[i] >= 0.96].index
        index2 = matrix[matrix[i] <= -0.96].index
        index = list(index1) + list(index2)
        for j in index:
            if j != i:
                tmp.append(j)
                cor.append(j)
        if len(tmp) != 0:
            matrix = matrix.drop(tmp, axis=0)
            matrix = matrix.drop(tmp, axis=1)
            tmp = []
print(len(cor))
data = data.drop(cor, axis=1)
print(data.info())

3501
<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Columns: 1470 entries, TOOL to Value
dtypes: float64(1285), int32(11), int64(174)
memory usage: 8.9+ MB
None


In [14]:
# 数据标准化
scaler = StandardScaler()
data_scaler = scaler.fit_transform(data)

print(data_scaler)

[[ 1.56685142  0.48289672  2.29628217 ... -0.97552246 -1.29294309
  -0.1004123 ]
 [ 1.56685142  0.48289672  0.74932334 ... -0.97552246 -1.29294309
  -0.25134847]
 [ 1.56685142  0.48289672  1.96921088 ... -0.97552246 -0.39197308
  -0.52815463]
 ...
 [ 0.79891992  0.52938975  0.32501463 ...  0.18254537  0.50899693
  -0.51406143]
 [-0.7369431   0.47585233  0.46645087 ...  0.18254537  1.40996693
   0.27219425]
 [ 0.79891992  0.52798087  0.25429651 ...  0.18254537  0.50899693
  -0.03987712]]


In [15]:
data = pd.DataFrame(data_scaler, columns=data.columns, index=data.index)

#输出训练集为excel文件
data.to_excel('Dataset/train.xlsx')

In [16]:
#输出测试集a为excel文件
data_a.to_excel('Dataset/testa.xlsx')

In [24]:
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor

data_result = data["Value"]
data_feature = data.drop(["Value"], axis=1)

print("feature")
selector = SelectFromModel(estimator=XGBRegressor(), threshold="6*mean")
selector.fit(data_feature, data_result)
feature_index = selector.get_support(indices=True)
print(feature_index)
feature_results = []
feature_names = data_feature.columns
for feature_index in feature_index:
    feature_results.append(feature_names[feature_index])
print(feature_results)
print(len(feature_results))

feature
[  14   42   45  110  128  137  155  246  258  266  274  280  281  325
  331  334  336  342  589  594  603  635  670  676  689  735  822  829
  837  850  856  943  952  959  961  964  969  992 1012 1023 1118 1120
 1191 1197 1229 1236 1240 1251 1255 1260 1327 1362 1369 1387 1398 1412
 1423 1426 1430]
['210X16', '210X76', '210X82', '210X231', '220X33', '220X163', '220X189', '311X4', '311X25', '311X52', '311X71', '311X86', '311X89', '311X154', '311X182', '311X194', '311X196', '311X208', '312X632', '312X670', '312X698', '330X61', '330X214', '330X261', '330X361', '330X594', '340X109', '340X125', '340X151', '344X1', '344X20', 'ERROR:#N/A_25', '360X58', '360X131', '360X155', '360X185', '360X252', '360X611', '360X995', '360X1245', '420X14', '420X19', '420X188', '420X205', '520X1', '520X18', '520X33', '520X61', '520X68', '520X78', '520X297', '520X399', '750X1', '750X155', '750X307', '750X508', '750X611', '750X635', '750X661']
59


In [25]:
data_dev = data

for i in data_dev.columns:
    if i not in feature_results:
        data_dev = data_dev.drop(i, axis=1)

print(data_dev.info())

<class 'pandas.core.frame.DataFrame'>
Index: 793 entries, NH0995 to NH1942
Data columns (total 59 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   210X16         793 non-null    float64
 1   210X76         793 non-null    float64
 2   210X82         793 non-null    float64
 3   210X231        793 non-null    float64
 4   220X33         793 non-null    float64
 5   220X163        793 non-null    float64
 6   220X189        793 non-null    float64
 7   311X4          793 non-null    float64
 8   311X25         793 non-null    float64
 9   311X52         793 non-null    float64
 10  311X71         793 non-null    float64
 11  311X86         793 non-null    float64
 12  311X89         793 non-null    float64
 13  311X154        793 non-null    float64
 14  311X182        793 non-null    float64
 15  311X194        793 non-null    float64
 16  311X196        793 non-null    float64
 17  311X208        793 non-null    float64
 18  312X632

In [26]:
data_dev.to_excel('Dataset/train_dev.xlsx')